# Libraries

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from config import *
import pandas as pd
import numpy as np
import pickle

# Importing DB

In [6]:
clustered_df = pd.read_csv('D:\Data Science\Ironhack\Week 3\Project_song_recomender\data\clustered_df.csv')
clustered_df.drop(['Unnamed: 0'], axis=1, inplace=True)
clustered_dF

,title,artist,cluster,origin,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,butter,bts,5,1,0.996282,-1.231372,0.749224,0.453165,0.780511,0.115918,-0.685853,-0.408605,-0.649467,0.762846,-0.488981,-1.277501
1,good 4 u,olivia rodrigo,1,1,-0.238658,-0.221006,1.026348,0.495894,0.780511,0.812795,0.639010,-0.408605,-0.685018,0.733866,1.613560,-1.000138
2,levitating,dua lipa featuring dababy,2,1,0.674946,0.429571,0.194977,0.562526,-1.281212,-0.281961,-0.663371,-0.408605,-0.797910,1.706778,-0.746429,-0.495866
3,peaches,justin bieber featuring daniel caesar & giveon,5,1,0.479624,-0.063290,-1.467764,0.156160,0.780511,0.400790,0.583103,-0.408605,1.405033,-0.193506,-1.226390,-0.596692
4,leave the door open,silk sonic (bruno mars & anderson .paak),5,1,-0.093742,-0.457580,-0.082146,-0.376597,0.780511,-0.618628,0.028032,-0.408605,-0.636369,0.862208,0.917772,0.294066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10038,yayo,lana del rey,0,0,-0.597799,-2.502955,-0.636394,-1.860727,-1.281212,-0.653942,2.999059,-0.406826,0.363438,-1.514182,-1.895032,1.892025
10039,yayo,lana del rey,0,0,-0.597799,-2.502955,-0.636394,-1.860727,-1.281212,-0.653942,2.999059,-0.406826,0.363438,-1.514182,-1.895032,1.892025
10040,regret in your tears,nicki minaj,0,0,1.065590,-1.142657,-1.467764,-1.275381,-1.281212,-0.055947,2.292242,0.491767,-0.179190,-1.087757,-0.631868,0.040625
10041,oblivion - anderex remix,riot shift,6,0,-1.713026,1.321651,0.749224,0.602266,0.780511,3.979346,-0.693959,-0.372634,0.644107,-0.930435,1.395258,-0.302208


# Defining functions

In [27]:
def load(filename = ""): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
    except FileNotFoundError: 
        print("File not found!") 

def what_is_the_cluster_of_the_song(song_name, song_artist):
    
    # connecting to the spotify db

    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID, client_secret= Client_Secret))
    
    # searching for the song

    result = sp.search(q="track:"+song_name+" artist:"+song_artist,limit=1)

    # obtaining uri

    song_uri = result['tracks']['items'][0]['uri']
    song_name = result['tracks']['items'][0]['name']
    song_artist = result['tracks']['items'][0]['artists'][0]["name"]

    # obtaining audio features

    song_features = sp.audio_features(song_uri)
    
    # cast audioe features to df

    song_features_df = pd.DataFrame(song_features)
    
    song_features_df = song_features_df[['danceability', 'energy', 'key', 'loudness', 'mode', 
                                'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]
    
    # open scaler and kmeans

    scaler = load('D:\Data Science\Ironhack\Week 3\Project_song_recomender\model\scaler.pickle')
    kmeans = load("D:\Data Science\Ironhack\Week 3\Project_song_recomender\model\Kmean.pickle")

    # scaling the song
    scaled_song_features_df = scaler.transform(song_features_df)
    
    predicted_value = kmeans.predict(scaled_song_features_df)
    
    return int(predicted_value),song_name, song_artist


In [68]:
def main(clustered_df):

    user_song = input("Tell me your favourite song! ")

    if user_song.lower() in clustered_df.loc[clustered_df['origin'] == 1, 'title'].str.lower().to_list():
        
        print('You are on the top of the wave: ')
        
        user_cluster = int(
            clustered_df.loc[clustered_df['title'] == user_song, 'cluster'])
        
        user_recommendation = clustered_df[(clustered_df['origin'] == 1) & (
            clustered_df['cluster'] == user_cluster)].sample()
        
        print('There is our recommendation: ' + list(user_recommendation['title'])[
              0] + ' played by ' + list(user_recommendation['artist'])[0])
        
    else:
        
        user_artist = input("Who is the singer? ")
        
        user_cluster, song_name, song_artist = what_is_the_cluster_of_the_song(
            user_song, user_artist)
        
        user_recommendation = clustered_df[clustered_df['cluster'] == user_cluster].sample(
        )
        
        print('There is our recommendation: ' + list(user_recommendation['title'])[
              0] + ' played by ' + list(user_recommendation['artist'])[0])

# Action!

## If song is not in hot top 100

In [69]:
main(clustered_df)

Tell me your favourite song! in the end
Who is the singer? linkin park
There is our recommendation: make it easy played by 3oh!3


## If song is in hot top 100

In [70]:
main(clustered_df)

Tell me your favourite song! butter
You are on the top of the wave: 
There is our recommendation: kiss me more played by doja cat featuring sza
